- 최종 데이터 정리본(루트에 들어가는 장소)
- 변수명
  - 장소명
  - 장소 분류
  - 장소별 주소
  - 장소별 탄소배출량
    - 숙소를 제외한 나머지 장소들은 1시간당 탄소배출량*머무는 시간
  - 각 장소별 머무는 시간(숙소는 default로 0 : 총 외출시간이 구해져야 나올 수 있는 값)
  - 비용

In [1]:
import os
os.chdir("C:/Users/daess/Desktop/study/공모전/2022 빅콘테스트/Data/Using Data")

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 필요한 패키지 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyautogui
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [4]:
# 경유지 포함할 경우의 이동거리와 이동시간 구하는 함수
# 이 함수를 실행할 때마다 실시간 정보를 반영한 결과 값이 나오기 때문에 매번 다른 결과가 나올 수 있습니다
def stopover(route):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://map.naver.com/v5/directions/-/-/-/mode?c=14107103.1786139,4494701.9630842,15,0,0,0,dh')
    delay = 3
    driver.implicitly_wait(delay)
    driver.find_element(By.CLASS_NAME, 'link_tab.car').click()
    driver.implicitly_wait(5)

    # 경유지 추가
    routelen = len(route)
    for _ in range(1, routelen-1):
        driver.find_element(By.CLASS_NAME, 'btn.btn_route.ng-star-inserted').click()

    # 출발지 입력
    driver.implicitly_wait(5)
    driver.find_element(By.ID,'directionStart0').click()
    driver.implicitly_wait(5)
    driver.find_element(By.ID,'directionStart0').send_keys(route[0])
    sleep(0.02)
    pyautogui.press('enter')
    sleep(0.3)

    # 경유지 입력
    for i in range(1, routelen - 1):
        via_id = 'directionVia' + str(i)
        driver.find_element(By.ID, via_id).send_keys(route[i])
        sleep(0.02)
        pyautogui.press('enter')
        sleep(0.3)

    # 도착지 입력
    goal_id = 'directionGoal' + str(routelen - 1)
    driver.find_element(By.ID, goal_id).send_keys(route[routelen - 1])
    sleep(0.02)
    pyautogui.press('enter')
    sleep(0.3)

    # 길찾기 버튼 클릭
    driver.find_element(By.CLASS_NAME, 'btn.btn_direction').send_keys(Keys.ENTER)
    sleep(0.5)

    # 시간, 거리 계산   
    time = driver.find_element(By.CLASS_NAME, 'duration_time').text
    distance = float(driver.find_element(By.CLASS_NAME, 'summary_info.distance').text.replace('km', ''))
    if '시간' in time:
        time = re.sub("[^0-9]", "", time)
        new_time = int(time[0])*60 + int(time[1:3])
    else:
        new_time = int(time.replace('분', ''))
    
    driver.close()
    
    return distance, new_time

In [5]:
# 전처리한 비짓제주 데이터 불러오기
visit_jeju = pd.read_csv('visit_jeju_fix_sample4_.csv')

In [6]:
visit_jeju.head()

,CL_NM,AREA_NM,ADDR,LTNO_ADDR,USE_MONTH,ALL_TOTAL_CO,SECTION,SUM_GRGS_DSAMT
0,관광지,1100고지(휴게소),서귀포시1100로1555,서귀포시색달동1-3번지,11.0,2679,2,276.610821
1,관광지,1100고지습지,서귀포시색달동산1-2,NaN,NaN,3860,2,NaN
2,관광지,1112도로,제주시명림로584,제주시봉개동78-1번지,3.0,6206,1,3226.512929
3,관광지,4·3해원방사탑,제주시신산로82,제주시일도이동837-20번지,9.0,280,1,306.031221
4,관광지,5.16도로숲터널,서귀포시남원읍신례리산2-1,NaN,NaN,2454,3,NaN


In [7]:
# 최종 정한 루트에 해당하는 장소들만 추출
# 장소 목록
loc_list = ['제주국제공항', '용두암', '고객식당', '새별오름', '협재해수욕장', '탐라반상', '틸다하우스',
            '용머리해안', '춘심이네본점', '천지연폭포', '쇠소깍', '어진이네횟집', '서귀포KAL호텔',
            '옷귀마테마타운', '섭지코지', '섭지해녀의집', '성산일출봉(UNESCO세계자연유산)', '우도(해양도립공원)', '가시아방', '더포그레이스호텔앤리조트',
            '비자림', '상춘재', '동문시장(주)', '제주국제공항']

final_visit = visit_jeju[visit_jeju['AREA_NM'].isin(loc_list)]

In [8]:
# 필요없는 변수 제거
final_visit.drop(['LTNO_ADDR', 'USE_MONTH', 'ALL_TOTAL_CO', 'SECTION'], axis = 1, inplace = True)

In [9]:
final_visit.head()

,CL_NM,AREA_NM,ADDR,SUM_GRGS_DSAMT
353,관광지,비자림,제주시구좌읍비자숲길55,NaN
395,관광지,새별오름,제주시애월읍봉성리산59-8,NaN
439,관광지,섭지코지,서귀포시성산읍섭지코지로107,NaN
448,관광지,성산일출봉(UNESCO세계자연유산),서귀포시성산읍일출로284-12,NaN
492,관광지,쇠소깍,서귀포시쇠소깍로128,190.618705


In [10]:
# 제주공항 제외
airport = final_visit[final_visit['AREA_NM']=='제주국제공항'].index
final_visit.drop(airport, inplace = True)

In [11]:
# 보기 쉽게 변수명 변경
final_visit.columns = ['장소 분류', '장소명', '주소', '탄소배출량(1일)']

In [12]:
# 장소별 머무는 시간 입력(우도 제외 나머지 모두 1시간으로 통일)
final_visit['체류 시간(분)'] = 60
final_visit.loc[final_visit.장소명 == '우도(해양도립공원)', '체류 시간(분)'] = 120

In [13]:
# 장소별 비용(만원)
final_visit['비용(만원)'] = 0
final_visit.loc[final_visit.장소명 == '비자림', '비용(만원)'] = 0.9
final_visit.loc[final_visit.장소명 == '쇠소깍', '비용(만원)'] = 2.4
final_visit.loc[final_visit.장소명 == '옷귀마테마타운', '비용(만원)'] = 18
final_visit.loc[final_visit.장소명 == '용두암', '비용(만원)'] = 3.15
final_visit.loc[final_visit.장소명 == '천지연폭포', '비용(만원)'] = 0.6
final_visit.loc[final_visit.장소명 == '더포그레이스호텔앤리조트', '비용(만원)'] = 7.2
final_visit.loc[final_visit.장소명 == '서귀포KAL호텔', '비용(만원)'] = 13
final_visit.loc[final_visit.장소명 == '틸다하우스', '비용(만원)'] = 18
final_visit.loc[final_visit.장소명 == '가시아방', '비용(만원)'] = 4.4
final_visit.loc[final_visit.장소명 == '고객식당', '비용(만원)'] = 5.3
final_visit.loc[final_visit.장소명 == '상춘재', '비용(만원)'] = 3.9
final_visit.loc[final_visit.장소명 == '섭지해녀의집', '비용(만원)'] = 3
final_visit.loc[final_visit.장소명 == '어진이네횟집', '비용(만원)'] = 8.1
final_visit.loc[final_visit.장소명 == '춘심이네본점', '비용(만원)'] = 11.5
final_visit.loc[final_visit.장소명 == '탐라반상', '비용(만원)'] = 4.5

In [14]:
# 주소 정보가 부족하여 주소 입력
final_visit.loc[final_visit.장소명 == '용머리해안', '주소'] = '서귀포시안덕면사계리112-3'

In [15]:
# 탄소배출량(1일)이 결측치인 값 채우기
# TB_ECO_BUILDING_GHG_GIS_JEJU_1920에서 추출
final_visit.loc[final_visit.장소명 == '옷귀마테마타운', '탄소배출량(1일)'] = 6.232213667
final_visit.loc[final_visit.장소명 == '더포그레이스호텔앤리조트', '탄소배출량(1일)'] = 169.1364841 / 144 # 객실 수로 나눔
final_visit.loc[final_visit.장소명 == '틸다하우스', '탄소배출량(1일)'] = 43.84307927 / 6 # 객실 수로 나눔
final_visit.loc[final_visit.장소명 == '가시아방', '탄소배출량(1일)'] = 3.458860233
final_visit.loc[final_visit.장소명 == '상춘재', '탄소배출량(1일)'] = 3.305098189
final_visit.loc[final_visit.장소명 == '섭지해녀의집', '탄소배출량(1일)'] = 3.458860233
final_visit.loc[final_visit.장소명 == '어진이네횟집', '탄소배출량(1일)'] = 5.336970332
final_visit.loc[final_visit.장소명 == '춘심이네본점', '탄소배출량(1일)'] = 3.474352666
final_visit.loc[final_visit.장소명 == '탐라반상', '탄소배출량(1일)'] = 86.59824097 / 2 # 2층 건물
final_visit.loc[final_visit.장소명 == '서귀포KAL호텔', '탄소배출량(1일)'] = 4635.274977 / 225 # 객실 수로 나눔

In [16]:
final_visit = final_visit.fillna(0)

In [17]:
# 네이버 길찾기 특성상 경유지가 최대 5개까지이므로 7개씩 나누기 
day_1 = ['제주국제공항', '용두암', '고객식당', '새별오름', '협재해수욕장', '탐라반상', '틸다하우스']
day_2 = ['틸다하우스', '용머리해안', '춘심이네본점', '천지연폭포', '쇠소깍', '어진이네횟집', '서귀포KAL호텔']
day_3 = ['서귀포KAL호텔', '옷귀마테마타운', '섭지코지', '섭지해녀의집', '성산일출봉(UNESCO세계자연유산)', '우도(해양도립공원)', '가시아방', '더포그레이스호텔앤리조트']
day_4 = ['더포그레이스호텔앤리조트', '비자림', '상춘재', '동문시장(주)', '제주국제공항']

In [18]:
all_days = [day_1, day_2, day_3, day_4]

In [19]:
all_times = []
all_dists = []
for day in all_days:
    day_addr = [] # 주소를 입력해야 정확한 위치를 찾기 때문에 주소 데이터 추출
    for location in day:
        if location == '제주국제공항':
            loc_addr = '제주국제공항'
        else:
            loc_addr = final_visit.loc[final_visit.장소명 == location, '주소'].iloc[0]
        day_addr.append(loc_addr)

    # 경유지는 최대 5개까지만 지정할 수 있으므로 출발지와 도착지를 포함하여 총 7개까지 입력 가능
    if len(day_addr) <= 7:
        day_dist, day_time = stopover(day_addr)
    else:
        half_dist1, half_time1 = stopover(day_addr[:7])
        half_dist2, half_time2 = stopover(day_addr[6:])
        day_dist = half_dist1 + half_dist2
        day_time = half_time1 + half_time2
    
    all_times.append(day_time)
    all_dists.append(day_dist)

In [20]:
times_dists = pd.DataFrame(zip(all_days, all_times, all_dists), columns = ['루트', '총 이동시간(분)', '총 이동거리(km)'])

In [21]:
times_dists

,루트,총 이동시간(분),총 이동거리(km)
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0


In [22]:
# 3일차에는 우도를 가기 때문에 배 이동시간, 이동거리 변수 생성
car_time, car_dist, boat_time, boat_dist = [], [], [], []

In [23]:
for i in range(4):
    if '우도(해양도립공원)' in all_days[i]:
        boat_time.append(30) # 왕복 30분
        boat_dist.append(5.4) # 왕복 5.4km
        car_time.append(times_dists.loc[i, '총 이동시간(분)'] - 30)
        car_dist.append(times_dists.loc[i, '총 이동거리(km)'] - 5.4)

    else:
        boat_time.append(0) # 왕복 30분
        boat_dist.append(0) # 왕복 5.4km
        car_time.append(times_dists.loc[i, '총 이동시간(분)'])
        car_dist.append(times_dists.loc[i, '총 이동거리(km)'])


In [24]:
times_dists['차 이동시간(분)'] = car_time
times_dists['차 이동거리(km)'] = car_dist
times_dists['배 이동시간(분)'] = boat_time
times_dists['배 이동거리(km)'] = boat_dist

In [25]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km)
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0


In [26]:
times_dists['총 외출시간(분)'] = 0

for i in range(4):
    day_out = 0
    for location in times_dists.loc[i, '루트']:
        if location == '제주국제공항':
            continue
        else:
            day_out += final_visit.loc[final_visit.장소명 == location, '체류 시간(분)'].iloc[0]
    times_dists.loc[i, '총 외출시간(분)'] = day_out + times_dists.loc[i, '총 이동시간(분)']
    

In [27]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km),총 외출시간(분)
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0,476
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0,523
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4,682
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0,323


In [28]:
# 숙소 
times_dists['숙소에 머무는 시간(분)'] = 0
start_time = datetime.strptime('10시 00분', '%H시 %M분')

for i in range(3): # 숙소에는 3일차까지만 있으니
    total_time = pd.to_timedelta(times_dists.loc[i, '총 외출시간(분)'], unit = 'm')
    arrive_time = start_time + total_time
    lodg_time = (start_time - arrive_time).seconds/60
    times_dists.loc[i, '숙소에 머무는 시간(분)'] = lodg_time

In [29]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km),총 외출시간(분),숙소에 머무는 시간(분)
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0,476,964
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0,523,917
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4,682,758
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0,323,0


In [69]:
final_visit

,장소 분류,장소명,주소,탄소배출량(1일),체류 시간(분),비용(만원)
353,관광지,비자림,제주시구좌읍비자숲길55,0.000000,60,0.90
395,관광지,새별오름,제주시애월읍봉성리산59-8,0.000000,60,0.00
439,관광지,섭지코지,서귀포시성산읍섭지코지로107,0.000000,60,0.00
448,관광지,성산일출봉(UNESCO세계자연유산),서귀포시성산읍일출로284-12,0.000000,60,0.00
492,관광지,쇠소깍,서귀포시쇠소깍로128,190.618705,60,2.40
596,관광지,옷귀마테마타운,서귀포시남원읍서성로955-117,6.232214,60,18.00
607,관광지,용두암,제주시용두암길15,32.562326,60,3.15
608,관광지,용머리해안,서귀포시안덕면사계리112-3,0.000000,60,0.00
612,관광지,우도(해양도립공원),제주시우도면삼양고수물길1,0.000000,120,0.00
847,관광지,천지연폭포,서귀포시남성중로2-15,285.718245,60,0.60


In [99]:
# 총 탄소배출량 계산
times_dists['총 탄소배출량'] = 0

for i in range(4):
    day_co2 = 0
    route = times_dists.loc[i, '루트']
    if '제주국제공항' in route:
        day_co2 += (0.12773818*438/189)*3 # (비행기 1km당 탄소배출량*(제주-김포 거리) / 비행기 수용인원)*3인
    for location in route[1:-1]:
        if location == '제주국제공항':
            continue
        else:
            hour = final_visit.loc[final_visit['장소명'] == location, '체류 시간(분)'].iloc[0]//60
            day_co2 += (final_visit.loc[final_visit['장소명'] == location, '탄소배출량(1일)'].iloc[0]/24)*hour
    
    if i < 3:
        lodg_hour = times_dists.loc[i, '숙소에 머무는 시간(분)'] / 60
        day_co2 += lodg_hour*(final_visit.loc[final_visit['장소명'] == location, '탄소배출량(1일)']/24)
        
    day_co2 += 0.09008*times_dists.loc[i, '차 이동거리(km)'] + 0.37760536*times_dists.loc[i, '배 이동거리(km)']

    if i < 3:
        times_dists.loc[i, '총 탄소배출량'] = day_co2.iloc[0]
    else:
        times_dists.loc[i, '총 탄소배출량'] = day_co2


In [100]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km),총 외출시간(분),숙소에 머무는 시간(분),총 탄소배출량,1인당 탄소배출량,총 비용
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0,476,964,44.899848,15.634080,60.95
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0,523,917,28.927844,10.267700,35.60
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4,682,758,11.821272,4.812365,32.60
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0,323,0,20.031306,7.280998,34.80


In [101]:
# 1인당 하루 탄소배출량 계산
# times_dists['CO2(per day person)']
# times_dists['total_CO2'] / 3(3인)
times_dists['1인당 탄소배출량'] = times_dists['총 탄소배출량']/3

In [102]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km),총 외출시간(분),숙소에 머무는 시간(분),총 탄소배출량,1인당 탄소배출량,총 비용
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0,476,964,44.899848,14.966616,60.95
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0,523,917,28.927844,9.642615,35.60
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4,682,758,11.821272,3.940424,32.60
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0,323,0,20.031306,6.677102,34.80


In [103]:
# 총 비용계산
times_dists['총 비용'] = 0

for i in range(4):
    day_cost = 0
    route = times_dists.loc[i, '루트']
    if '제주국제공항' in route:
        day_cost += 10*3
    for location in route[1:]:
        if location == '제주국제공항':
            continue
        else:
            day_cost += final_visit.loc[final_visit.장소명 == location, '비용(만원)'].iloc[0]
    
    times_dists.loc[i, '총 비용'] = day_cost

In [104]:
times_dists

,루트,총 이동시간(분),총 이동거리(km),차 이동시간(분),차 이동거리(km),배 이동시간(분),배 이동거리(km),총 외출시간(분),숙소에 머무는 시간(분),총 탄소배출량,1인당 탄소배출량,총 비용
0,"[제주국제공항, 용두암, 고객식당, 새별오름, 협재해수욕장, 탐라반상, 틸다하우스]",116,63.0,116,63.0,0,0.0,476,964,44.899848,14.966616,60.95
1,"[틸다하우스, 용머리해안, 춘심이네본점, 천지연폭포, 쇠소깍, 어진이네횟집, 서귀포...",103,59.0,103,59.0,0,0.0,523,917,28.927844,9.642615,35.60
2,"[서귀포KAL호텔, 옷귀마테마타운, 섭지코지, 섭지해녀의집, 성산일출봉(UNESCO...",142,87.7,112,82.3,30,5.4,682,758,11.821272,3.940424,32.60
3,"[더포그레이스호텔앤리조트, 비자림, 상춘재, 동문시장(주), 제주국제공항]",83,57.0,83,57.0,0,0.0,323,0,20.031306,6.677102,34.80


In [110]:
sum(times_dists['차 이동거리(km)'])

261.3

In [93]:
# 4일 총 이동거리
sum(times_dists['총 이동거리(km)'])

266.7

In [94]:
# 4일 총 탄소배출량
sum(times_dists['총 탄소배출량'])

113.98542954544541

In [105]:
round((131175 + 165674) / 2, 2)

296849

In [109]:
# 4일 총 비용
sum(times_dists['총 비용']) + 15.4174 + 0.81 # 렌터카 비용 + 급속 충전 비용

180.17739999999998